# Conversión audio-texto.

El documento contempla dos alternativas:
1. Google Speech to text
2. Vosk

## Google Speech-to-text

In [1]:
# Se contempla el uso de el SDK de Google y por tanto de sus credenciales.
#pip install google-cloud-speech

In [32]:
def transcribe_audio_with_diarization_and_punctuation_async(gcs_uri, output_txt_path):
    client = speech.SpeechClient()
    
    audio = speech.RecognitionAudio(uri=gcs_uri)
    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=16000,
        language_code="es-ES",
        enable_speaker_diarization=True,
        #model="video",  # Utilizar el modelo 'video'
        diarization_speaker_count=2,
        enable_automatic_punctuation=True
    )

    operation = client.long_running_recognize(config=config, audio=audio)
    response = operation.result(timeout=600)  # Aumentar el tiempo de espera si es necesario
    
    with open(output_txt_path, "w") as txt_file:
        for i, result in enumerate(response.results):
            alternative = result.alternatives[0]
            segment_text = f"Segmento {i+1}:\n"
            transcript_text = f"Transcripción: {alternative.transcript}\n"
            
            # Manejo de errores para el campo speaker_tag
            try:
                speaker_text = f"Interlocutor: {alternative.speaker_tag}\n"
            except AttributeError:
                speaker_text = "Interlocutor: Desconocido\n"
            
            print(segment_text)
            print(transcript_text)
            print(speaker_text)
            print()

            txt_file.write(segment_text)
            txt_file.write(transcript_text)
            txt_file.write(speaker_text)
            txt_file.write("\n")


In [33]:
# Uso de la función
gcs_uri = "gs://"
transcribe_audio_with_diarization_and_punctuation_async(gcs_uri, "Result.txt")


Segmento 1:

Transcripción: No, la conocí sería un poco respetuoso hablar sobre una persona que si existió o no, entonces es una inspirado en ese caso y solamente es eso lo que yo tomo en términos de lo que se ocurrió un término concreto, pero no es mía y tu familia por supuesto no tiene nada que ver con no tengo idea como la familia, pero no lo ejerzo obviamente, no? Entonces sí que fue un pie un pie forzado, puede ser como una inspiración o un claro exactamente un punto inicio.

Interlocutor: Desconocido


Segmento 2:

Transcripción:  Los religiosos sí, una cosa más bien. Te llamo mucho la atención es todo justo.

Interlocutor: Desconocido


Segmento 3:

Transcripción:  Perdona, disculpa ya, yo creo que es súper importante lo que tú comentas, porque se activa mente, hay una cuestión entorno, lo espiritual y religioso que todos con los que todos nos podemos identificar de una buena manera. Ya sea si tú tienes una YouTube forma parte de una religión convencional como el gato si formas 

## VOSK

In [38]:
from vosk import Model, KaldiRecognizer
import soundfile as sf
import json

# Cargar el modelo
model = Model("/Users/jan/Downloads/vosk-model-es-0.42")

# Configurar el reconocedor
rec = KaldiRecognizer(model, 16000)


LOG (VoskAPI:ReadDataFiles():model.cc:213) Decoding params beam=13 max-active=7000 lattice-beam=6
LOG (VoskAPI:ReadDataFiles():model.cc:216) Silence phones 1:2:3:4:5:6:7:8:9:10
LOG (VoskAPI:RemoveOrphanNodes():nnet-nnet.cc:948) Removed 0 orphan nodes.
LOG (VoskAPI:RemoveOrphanComponents():nnet-nnet.cc:847) Removing 0 orphan components.
LOG (VoskAPI:ReadDataFiles():model.cc:248) Loading i-vector extractor from /Users/jan/Downloads/vosk-model-es-0.42/ivector/final.ie
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:183) Computing derived variables for iVector extractor
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:204) Done.
LOG (VoskAPI:ReadDataFiles():model.cc:279) Loading HCLG from /Users/jan/Downloads/vosk-model-es-0.42/graph/HCLG.fst
LOG (VoskAPI:ReadDataFiles():model.cc:294) Loading words from /Users/jan/Downloads/vosk-model-es-0.42/graph/words.txt
LOG (VoskAPI:ReadDataFiles():model.cc:303) Loading winfo /Users/jan/Downloads/vosk-model-es-0.42/graph/phones/word_bound

In [44]:
from vosk import Model, KaldiRecognizer
import soundfile as sf
import json
import numpy as np

# Cargar el modelo
model = Model("/Users/jan/Downloads/vosk-model-es-0.42")

# Configurar el reconocedor
rec = KaldiRecognizer(model, 16000)

transcription_text = ""

# Leer el archivo de audio
with sf.SoundFile("/Users/jan/Desktop/audio_normalized.wav") as f:
    samplerate = f.samplerate
    rec = KaldiRecognizer(model, samplerate)
    while True:
        data = f.buffer_read(2000, dtype='int16')
        if len(data) == 0:
            break
        np_data = np.frombuffer(data, dtype=np.int16)
        if rec.AcceptWaveform(np_data.tobytes()):
            result_str = rec.Result()
            result_dict = json.loads(result_str)
            transcription = result_dict.get('text', '')
            transcription_text += transcription + " "

# Obtener el resultado final
result_str = rec.FinalResult()
result_dict = json.loads(result_str)
final_transcription = result_dict.get('text', '')
transcription_text += final_transcription

# Guardar en un archivo de texto
with open("/Users/jan/Desktop/vosk.txt", "w") as txt_file:
    txt_file.write(transcription_text)

print("Transcripción final almacenada en el archivo de texto.")


LOG (VoskAPI:ReadDataFiles():model.cc:213) Decoding params beam=13 max-active=7000 lattice-beam=6
LOG (VoskAPI:ReadDataFiles():model.cc:216) Silence phones 1:2:3:4:5:6:7:8:9:10
LOG (VoskAPI:RemoveOrphanNodes():nnet-nnet.cc:948) Removed 0 orphan nodes.
LOG (VoskAPI:RemoveOrphanComponents():nnet-nnet.cc:847) Removing 0 orphan components.
LOG (VoskAPI:ReadDataFiles():model.cc:248) Loading i-vector extractor from /Users/jan/Downloads/vosk-model-es-0.42/ivector/final.ie
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:183) Computing derived variables for iVector extractor
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:204) Done.
LOG (VoskAPI:ReadDataFiles():model.cc:279) Loading HCLG from /Users/jan/Downloads/vosk-model-es-0.42/graph/HCLG.fst
LOG (VoskAPI:ReadDataFiles():model.cc:294) Loading words from /Users/jan/Downloads/vosk-model-es-0.42/graph/words.txt
LOG (VoskAPI:ReadDataFiles():model.cc:303) Loading winfo /Users/jan/Downloads/vosk-model-es-0.42/graph/phones/word_bound

Transcripción final almacenada en el archivo de texto.


# Reflexiones

La API de Google tiene el importante plus de ser capaz de detectar interlocutores. Cuenta con compatibilidad para castellano de España y castellano de Chile, sin embargo, el primero posee muchas más herramientas disponibles, al mismo tiempo que detecta de mejor manera las palabras del audio. Por otra parte, Vosk mostró un mejor desempeño en la detección del audio, sin embargo, el resultado no presenta una estructura entre interlocutores ni distinguiendo párrafos.

# Anexo

## Conversión de M4A a WAV

In [20]:
from pydub import AudioSegment

def convert_m4a_to_wav(m4a_path, wav_path):
    audio = AudioSegment.from_file(m4a_path, format="m4a")
    audio = audio.set_channels(1).set_frame_rate(16000)  # Configurar para un solo canal y 16000 Hz
    audio.export(wav_path, format="wav")

# Uso de la función
convert_m4a_to_wav("/Users/jan/Desktop/Recortado.m4a", "/Users/jan/Desktop/Recortado.wav")



## Normalización del volumen 

In [35]:
from pydub import AudioSegment

# Cargar el archivo de audio
audio = AudioSegment.from_file("Recortado.wav")

target_dBFS = -20

# Normalizar al volumen deseado (dB)
normalized_audio = audio.apply_gain(-audio.dBFS + target_dBFS)

# Guardar el archivo de audio preprocesado
normalized_audio.export("audio_normalized.wav", format="wav")


<_io.BufferedRandom name='/Users/jan/Desktop/audio_normalized.wav'>